# Final Notebook for King County Housing Prices Report 

The purpose of this notebook is to layout the steps that our group took for the following parts of this project:
 - Data acquisition
 - Data cleaning/exploration
 - Modeling
 - Statistical Testing
 
Included are all of the necessary code cells needed to recreate our project.

## Data Acquisition

Our data came from the King County website [here](https://info.kingcounty.gov/assessor/DataDownload/default.aspx). From this site we downloaded three .csv files that would make up the entirety of our dataset for the project. These were 
 1. Real Property Sales
 2. Residential Building
 3. Parcel

With these three datasets, along with the provided Lookup table, we were able to get a lot of information on each property sale. The datasets were far too large to save and pass around intially; to make them easier to deal with, we imported them, joined them using their common Major and Minor PIN columns, and then filtered only those sales from 2019. We exported them into the data directory as 'master.csv', which we will import here